In [ ]:
#from numpy.core.numeric import infty
!pip install numpy
!pip install pandas
!pip install plotly
!pip install scikit-learn # non-depreceated sklearn

TASK 1

In [ ]:
import pandas as pd
# read the csv and make it a pandas.DataFrame
frame = pd.DataFrame(pd.read_csv("./blood_transfusion.csv"))
frame.describe(include='all') # 5 columns, 4 numerical, class is nominal (1 or 0)

In [ ]:
import plotly.express as px

"""fig = px.histogram(frame.loc[frame['class'] == 1].Margin, x="Margin", title="Histogram of Margin with Severity = 1")
fig.show()"""

#2.3
# A pie-chart showing how many cases of each class exist
fig = px.pie(frame, values='months_since_last_donation', names='class', title='% of classes')
fig.show()
# Duidelijk verschil tussen het percentage 0 en 1 wanneer mensen een kortere periode hebben sinds laatste donatie
fig = px.histogram(frame, x='months_since_last_donation',  color='class', title='')
fig.show()
# percentage redelijk gelijk over alle data
fig = px.histogram(frame, x='months_since_first_donation',  color='class', title='')
fig.show()
# Meer donations is gelijk aan hoger percentage class 1
fig = px.histogram(frame, x='total_number_of_donations',  color='class', title='')
fig = px.scatter(frame, x='months_since_last_donation', y="total_number_of_donations", color='class', title='')
fig.show()
fig = px.scatter(frame, x='months_since_last_donation', y="total_number_of_donations", color='class', title='')
fig.show()
fig = px.histogram(frame, x='total_blood_donated',  color='class', title='')
fig.show()
fig = px.scatter(frame, x='months_since_last_donation', y="months_since_first_donation", color='class', title='')
fig.show()

# total blood donated is linear with number of donations: dus wss niet nodig om beide te hebben in een vergelijking
fig = px.scatter(frame, x='total_blood_donated', y="total_number_of_donations", color='class', title='')
fig.show()
fig = px.scatter_3d(frame, x="months_since_last_donation", y="total_blood_donated", z='total_number_of_donations', color="class", title="3d scatter")
fig.show()

# Months since last donation is belangrijkst, spreid het meeste data, gevolgd door total blood donated. Miss een importance checker hier
fig = px.scatter_3d(frame, x="months_since_last_donation", y="total_blood_donated", z='months_since_first_donation', color="class", title="3d scatter")
fig.show()

fig = px.scatter(frame, x='months_since_last_donation', y='months_since_first_donation', color='class', title="scatter")
fig.show()

fig = px.scatter(frame, x='months_since_first_donation', y='months_since_last_donation', color='class', title="scatter")
fig.show()
fig = px.histogram(frame, x='months_since_last_donation', color='class', title='')
fig.show()

TASK 2

In [ ]:
preprocessed = frame.copy() 
preprocessed = (preprocessed-preprocessed.min())/(preprocessed.max()-preprocessed.min()) # Normalization
# It also normalizes the class, but because class is either 1 or 0, the formula does not change the class 
preprocessed.describe()

TASK 3

In [ ]:
from sklearn.model_selection import train_test_split
X_train_s, X_test_s, Y_train_s, Y_test_s = train_test_split(
    preprocessed.drop('class', axis=1), preprocessed['class'], train_size=0.33, test_size=0.33, random_state=1) # small test_size
X_train_l, X_test_l, Y_train_l, Y_test_l = train_test_split(
    preprocessed.drop('class', axis=1), preprocessed['class'], train_size=0.33, test_size=0.66, random_state=1) # large test_size
# random state for reproducibility

In [ ]:
""""NIET MET DE OPDRACHT TE MAKEN, FEATURE IMPORTANCE VOOR DE PRESENTATIE"""

import pandas as pd

from sklearn.ensemble import RandomForestClassifier
names = (list(X_train_s))
feature_names = [f"feature {names[i]}" for i in range(X_train_s.shape[1])]
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train_s, Y_train_s)
import time
import matplotlib.pyplot as plt
import numpy as np

start_time = time.time()
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")
forest_importances = pd.Series(importances, index=feature_names)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

TASK 4 & 5

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score

#5.1 Confusion matrix
def confusionmatrix(predicted_y, real_y): # it works, im confused :) \j
    """
    This function calculates and prints the confusion matrix.
    :param predicted_y: list: predicted classes for the test data 
    :param real_y: list: real classes of the test data
    :return: prints the confusion matrix, returns nothing
    """
    tp, fp, tn, fn = 0, 0, 0, 0
    for index,_ in enumerate(predicted_y):
        if predicted_y[index][0] == 1: 
            if predicted_y[index] == real_y[index]: 
                tp += 1
            else:
                fp += 1
        elif predicted_y[index][0] == 0:
            if predicted_y[index] == real_y[index]: 
                tn += 1
            else:
                fn += 1
    print(f"\t P \t\t N \n T \t {tp} \t\t {tn} \n F \t {fp} \t\t {fn} \n")

def accuracy_checker(X_test, y_test, pred, mode="basic"):
    """
    This function checks the accuracy of the predicted classes against the real classes.
    :param X_test: pandas.DataFrame: the test data to be classified
    :param y_test: pandas.Series: the real classes of the test data
    :param pred: predictor class: the method used to classify the test data
    :param mode: string: way to evaluate the results
    :return: prints the results, returns nothing
    """
    num, total = 0, 0
    predicted, true = [], []
    for index, row in X_test.iterrows():
        p = pred.predict(X_test.loc[[index]])
        q = list(y_test.loc[[index]])
        if p == q:
            num += 1
        total += 1
        predicted.append(p)
        true.append(q)
    if mode == "basic":
        print(f"Accuracy: {num/total*100:.2f}%")
    elif mode == "classification_report":
        print(classification_report(true, predicted))
    elif mode == "confusion_matrix":
        confusionmatrix(predicted, true)
    elif mode == "fbeta":
        print("average=macro:", fbeta_score(true, predicted, average='macro', beta=0.5))
        print("average=weighted:", fbeta_score(true, predicted, average='weighted', beta=0.5))
        print("average=None:", fbeta_score(true, predicted, average=None, beta=0.5))
    else:
        return num/total*100

# 4.1 KNN
class KNN:    
    def __init__(self, k=5):
        """
        initializes an instance of the KNN class
        :param k: int: number of neighbors
        """
        self.k = k
        self.X_train = None
        self.y_train = None
        
    def fit(self, X, y):
        """
        fits the KNN class
        :param X: pandas.DataFrame: the train data
        :param y: pandas.Series: the classes of the train data
        :return: a fitted instance of the KNN class
        """
        self.X_train = X
        self.y_train = y
        return self
    
    def euclidean_distance(self, a, b):
        """
        This function calculates the euclidean distance between two datapoints.
        :param a: list: the first datapoint
        :param b: list: the second datapoint
        :return: float: the euclidean distance between the datapoints
        """
        distance = 0
        for index, _ in enumerate(a):
            distance += (a[index] - b[index]) ** 2
        distance = distance**0.5
        return distance    
    
    def predict(self, X_test):
        """
        predicts the class of the test data
        :param X_test: pandas.DataFrame: the test data
        :return: list: the predicted classes
        """
        test_y = list([0 for _ in range(X_test.shape[0])])
        for index, (_,test_row) in enumerate(X_test.iterrows()):
            distances = []
            for _, train_row in self.X_train.iterrows():
                test_values = list(test_row)  
                train_values = list(train_row)
                distance = self.euclidean_distance(test_values, train_values)
                distances.append(distance)
            nearest_neighbours = sorted(range(len(distances)), key=lambda i: distances[i])[:self.k]
            neighbor_labels = self.y_train.iloc[nearest_neighbours]
            predicted_label = round(neighbor_labels.mean())
            test_y[index] = predicted_label
        return test_y

# from accuracy it appears that k=5 is best for small set and k=[2,4,9] are all best for large set, for 1<=k<=9
knn_s = KNN(5)
knn_s.fit(X_train_s, Y_train_s)
knn_s_acc = accuracy_checker(X_test_s, Y_test_s, knn_s, "return")
knn_l = KNN(5)
knn_l.fit(X_train_l, Y_train_l)
knn_l_acc = accuracy_checker(X_test_l, Y_test_l, knn_l, "return")

# 4.2 nbc
from sklearn.naive_bayes import GaussianNB
nbc_s = GaussianNB()
nbc_s.fit(X_train_s, Y_train_s)
nbc_s_acc = accuracy_checker(X_test_s, Y_test_s, nbc_s, "return")
nbc_l = GaussianNB()
nbc_l.fit(X_train_l, Y_train_l)
nbc_l_acc = accuracy_checker(X_test_l, Y_test_l, nbc_l, "return")

# 4.3 svc
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
svc_s = make_pipeline(StandardScaler(), SVC(gamma='auto'))
svc_s.fit(X_train_s, Y_train_s)
svc_s_acc = accuracy_checker(X_test_s, Y_test_s, svc_s, "return")
svc_l = make_pipeline(StandardScaler(), SVC(gamma='auto'))
svc_l.fit(X_train_l, Y_train_l)
svc_l_acc = accuracy_checker(X_test_l, Y_test_l, svc_l, "return")

# 4.4 mlp
from sklearn.neural_network import MLPClassifier
mlp_s = MLPClassifier(random_state=1, max_iter=400).fit(X_train_s, Y_train_s)
mlp_s_acc = accuracy_checker(X_test_s, Y_test_s, mlp_s, "return")
mlp_l = MLPClassifier(random_state=1, max_iter=400).fit(X_train_l, Y_train_l)
mlp_l_acc = accuracy_checker(X_test_l, Y_test_l, mlp_l, "return")

classifiers_list = ["knn_s", "knn_l", "nbc_s", "nbc_l", "svc_s", "svc_l", "mlp_s", "mlp_l"]
accuracy_list = [knn_s_acc, knn_l_acc, nbc_s_acc, nbc_l_acc, svc_s_acc, svc_l_acc, mlp_s_acc, mlp_l_acc]
fig = px.histogram(x=classifiers_list, y=accuracy_list, barmode='group', height=500, title='Classifiers and accuracy', text_auto=True)
fig.show()


In [ ]:
preds = [knn_s, knn_l, nbc_s, nbc_l, svc_s, svc_l, mlp_s, mlp_l]
modes = ["confusion_matrix", "classification_report", "fbeta"]
for mode in modes:
    for pred in preds:
        pred_string = ""
        if pred == svc_s: pred_string = "svc_s"
        elif pred == svc_l: pred_string = "svc_l"
        elif pred == mlp_s: pred_string = "mlp_s"
        elif pred == mlp_l: pred_string = "mlp_l"
        elif pred == nbc_s: pred_string = "nbc_s"
        elif pred == nbc_l: pred_string = "nbc_l"
        elif pred == knn_s: pred_string = "knn_s"
        elif pred == knn_l: pred_string = "knn_l"
        if pred_string[4] == "s":
            print(f"{mode} for {pred_string}")
            accuracy_checker(X_test_s, Y_test_s, pred, mode=mode)    
        elif pred_string[4] == "l":
            print(f"\n{mode} for {pred_string}")
            accuracy_checker(X_test_l, Y_test_l, pred, mode=mode)
        print("-----------------------------------------")

TASK 6

In [ ]:

from sklearn.model_selection import KFold
import random
X = preprocessed.copy()

random_tester = []
for b in range(0,8):
    kf = KFold(n_splits=3, shuffle=True, random_state=random.randint(0,10000))
    kf.get_n_splits()

    print(kf)
    folds = []
    KFold(n_splits=3, random_state=None, shuffle=False)
    for (train_index, test_index) in (kf.split(X)):
        folds.append((train_index, test_index))

    accuracy = []
    parameters = [2,4,5,9]

    for m, (i,j) in enumerate(folds):
        temptrainx = X.iloc[list(i)].drop('class', axis=1)
        temptrainy = X.iloc[list(i)]['class']
        temptestx = X.iloc[list(j)].drop('class', axis=1)
        temptesty = X.iloc[list(j)]['class']

        knn_s = KNN(parameters[m])
        knn_s.fit(temptrainx, temptrainy)
        accuracy.append(accuracy_checker(temptestx, temptesty, knn_s,mode="basicRET"))

    print(max(accuracy), f"Best number of neighbors is: {parameters[accuracy.index(max(accuracy))]}")
    random_tester.append(parameters[accuracy.index(max(accuracy))])
print(random_tester)

In [ ]:

# visualisation

kf = KFold(n_splits=3, shuffle=False)
kf.get_n_splits()

print(kf)
folds = []
KFold(n_splits=3, random_state=None, shuffle=False)
for (train_index, test_index) in (kf.split(X)):
    folds.append((train_index, test_index))
randomint = max(set(random_tester), key=random_tester.count)
parameters =  [randomint for i in range(1,4)]
accuracy = []

for m, (i,j) in enumerate(folds):
    temptrainx = X.iloc[list(i)].drop('class', axis=1)
    temptrainy = X.iloc[list(i)]['class']
    temptestx = X.iloc[list(j)].drop('class', axis=1)
    temptesty = X.iloc[list(j)]['class']

    knn_s = KNN(parameters[m])
    knn_s.fit(temptrainx, temptrainy)
    accuracy.append(accuracy_checker(temptestx, temptesty, knn_s, mode="return"))
print(max(accuracy), f"Best number of neighbors is: {parameters[accuracy.index(max(accuracy))]}")

fig = px.bar(x=[x for x in range(1,len(folds)+1)], y= accuracy, title='Accuracy per fold', text_auto=True)
fig.update_layout(yaxis_title="accuracy", xaxis_title="")
fig.show()